## How to handle outlier for linear regression

In [ ]:
Excellent — handling outliers is especially important for linear regression, because it’s very sensitive to extreme values.
Let’s go step-by-step 👇

## 🧭 Step 1: Understand Why Outliers Matter

Outliers can:

* Distort the regression line, heavily influencing the slope and intercept.

* Inflate error terms (like MSE).

* Violate assumptions of linear regression (normality, homoscedasticity).

So, identifying and treating them properly helps your model stay robust.

## 🧩 Step 2: Detect Outliers
🔹 1. Using Boxplots (IQR Method)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x=df['feature'])
plt.show()


In [ ]:
You can also compute outliers programmatically:

In [ ]:
Q1 = df['feature'].quantile(0.25)
Q3 = df['feature'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['feature'] < lower_bound) | (df['feature'] > upper_bound)]


In [ ]:
🔹 2. Using Z-score (Standard Deviation Method)

In [ ]:
from scipy import stats
import numpy as np

z = np.abs(stats.zscore(df['feature']))
df[z > 3]  # Outliers where |z| > 3


In [ ]:
🔹3. Using Scatter Plots / Residual Plots
Outliers in predictor–response relationships can be visualized:

In [ ]:
sns.scatterplot(x='feature', y='target', data=df)
pt.show()

In [ ]:
After fitting a model, check residuals:

In [ ]:
import statsmodels.api as sm

model = sm.OLS(df['target'], sm.add_constant(df['feature'])).fit()
residuals = model.resid
sns.scatterplot(x=df['feature'], y=residuals)
plt.axhline(0, color='red')
plt.show()


In [ ]:
Points far from the main cluster are potential outliers or high-leverage points.

## 🧠 Step 3: Handle Outliers (Choose a Strategy)
🔹 1. Remove Outliers

If they’re clearly erroneous (e.g., data entry errors):

In [ ]:
df = df[(df['feature'] >= lower_bound) & (df['feature'] <= upper_bound)]


In [ ]:
✅ Simple
⚠️ Risky if outliers are actually valid extreme observations.

In [ ]:
🔹 2. Cap or Floor (Winsorization)

Limit extreme values to a reasonable percentile range.

In [ ]:
upper_cap = df['feature'].quantile(0.95)
lower_cap = df['feature'].quantile(0.05)

df['feature'] = np.clip(df['feature'], lower_cap, upper_cap)


In [ ]:
✅ Keeps data size intact
⚠️ Still distorts distribution slightly

In [ ]:
🔹 3. Transform the Variable

Use transformations to reduce the effect of outliers.

* Log transform

In [ ]:
df['feature_log'] = np.log1p(df['feature'])


In [ ]:
* Square root or Box-Cox transforms

In [ ]:
from scipy.stats import boxcox
df['feature_boxcox'], _ = boxcox(df['feature'] + 1)


In [ ]:
✅ Reduces skewness and impact of large values

In [ ]:
🔹 4. Use Robust Models or Techniques
If outliers can’t be removed:

* Robust Regression (less sensitive to outliers)

In [ ]:
from sklearn.linear_model import HuberRegressor

model = HuberRegressor().fit(X, y)


In [ ]:
* RANSAC Regression (fits model to inliers only)

In [ ]:
from sklearn.linear_model import RANSACRegressor, LinearRegression

base_model = LinearRegression()
ransac = RANSACRegressor(base_model).fit(X, y)


In [ ]:
✅ Resistant to outliers
⚠️ Slightly more complex to tune

In [ ]:
🔹 5. Apply Robust Scaling (Preprocessing)

Standard scaling can be distorted by outliers; use RobustScaler instead.

In [ ]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df[['feature1', 'feature2']] = scaler.fit_transform(df[['feature1', 'feature2']])


In [ ]:
✅ Scales features based on median & IQR
⚠️ Doesn’t remove outliers, just minimizes their influence

## 🧾 Step 4: Validate After Handling

* Recheck distributions or boxplots.

* Refit your regression and inspect:

* R² (fit quality)

* Residual plots

* Cook’s distance (to detect influential points)

In [ ]:
influence = model.get_influence()
cooks = influence.cooks_distance[0]


In [ ]:
influence = model.get_influence()
cooks = influence.cooks_distance[0]


## ⚙️ Recommended Workflow Summary

In [ ]:
| Step | Action                                                          |
| ---- | --------------------------------------------------------------- |
| 1    | Detect outliers with IQR, Z-score, or residual plots            |
| 2    | Investigate if they’re valid or errors                          |
| 3    | Treat using removal, winsorization, or transformation           |
| 4    | Consider robust regression if outliers are real but influential |
| 5    | Validate model stability and assumptions                        |
